In [6]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader

# Define dataset transformation with Grayscale to RGB conversion
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert 1-channel grayscale to 3-channel RGB
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Caltech-101 dataset
train_dataset = datasets.Caltech101(root="./data", download=True, transform=transform)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load Pretrained ResNet-50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)

# Modify the last FC layer for 101 classes
model.fc = nn.Linear(model.fc.in_features, 101)
model = model.to(device)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Training function
def train_model(model, dataloader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss, correct, total = 0, 0, 0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}, Accuracy: {correct/total:.4f}")

# Train before pruning
print("Training Base Model...")
train_model(model, train_loader, optimizer, criterion, epochs=5)

# ----------------- STRUCTURED PRUNING -----------------

def prune_model(model, amount=0.3):
    """
    Apply structured pruning (L1 Norm based) on convolutional layers.
    Removes `amount` of least important filters per layer.
    """
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)  # L1 Norm Pruning
            prune.remove(module, 'weight')  # Make pruning permanent
    return model

# Apply structured pruning (30% of filters)
print("Applying Structured Pruning...")
pruned_model = prune_model(model, amount=0.3)

# Fine-tune after pruning
print("Fine-tuning Pruned Model...")
train_model(pruned_model, train_loader, optimizer, criterion, epochs=5)

# Evaluate the model
def evaluate_model(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Evaluate on test set
accuracy = evaluate_model(pruned_model, val_loader)
print(f"Final Pruned Model Accuracy: {accuracy:.4f}")


Files already downloaded and verified
Training Base Model...
Epoch 1, Loss: 1.4322, Accuracy: 0.7181
Epoch 2, Loss: 0.2216, Accuracy: 0.9543
Epoch 3, Loss: 0.0792, Accuracy: 0.9833
Epoch 4, Loss: 0.0330, Accuracy: 0.9932
Epoch 5, Loss: 0.0152, Accuracy: 0.9978
Applying Structured Pruning...
Fine-tuning Pruned Model...
Epoch 1, Loss: 0.6309, Accuracy: 0.8457
Epoch 2, Loss: 0.1232, Accuracy: 0.9723
Epoch 3, Loss: 0.0515, Accuracy: 0.9891
Epoch 4, Loss: 0.0343, Accuracy: 0.9925
Epoch 5, Loss: 0.0248, Accuracy: 0.9951
Final Pruned Model Accuracy: 0.9361
